In [1]:
import numpy as np
import pandas as pd

import cobra
from cameo import pfba
from cameo.flux_analysis.simulation import room, lmoma

import escher

In [2]:
model = cobra.io.load_json_model('../../../DataAnalysis/DataIntegrationProject/models/iML1515.json')

## Define IDs of knocked-out reactions

In [3]:
knockouts = [
    {"gene": "fbaA", "id": "FBA"},
    {"gene": "fbaB", "id": "FBA"},
    {"gene": "fbp", "id": "FBP"},
    {"gene": "gnd", "id": "GND"},
    {"gene": "pfkA", "id": "PFK"},
    {"gene": "pfkB", "id": "PFK"},
    {"gene": "pgi", "id": "PGI"},
    {"gene": "pgl", "id": "PGL"},
    {"gene": "ppsA", "id": "PPS"},
    {"gene": "pts", "id": "GLCptspp"},
    {"gene": "pykA", "id": "PYK"},
    {"gene": "pykF", "id": "PYK"},
    {"gene": "rpe", "id": "RPE"},
    {"gene": "rpiA", "id": "RPI"},
    {"gene": "rpiB", "id": "RPI"},
    {"gene": "sdhCD", "id": "SUCDi"},
    {"gene": "sucAB", "id": "AKGDH"},
    {"gene": "talA", "id": "TALA"},
    {"gene": "tktAB", "id": "TKT1"},
    {"gene": "tktB", "id": "TKT1"},
    {"gene": "tpi", "id": "TPI"},
    {"gene": "zwf", "id": "G6PDH2r"},
]

## Batch simulation

In [13]:
pfba_result = pfba(model)

In [8]:
with model:
    model.reactions.G6PDH2r.knock_out()
    knockout_result = pfba(model)
    lmoma_result = lmoma(model, reference=pfba_result.fluxes)

In [14]:
escher.Builder(
    map_name="e_coli_core.Core metabolism",
    model=model,
    reaction_data=pfba_result.fluxes.to_dict(),
).display_in_browser()


Note: You must interrupt the kernel to end this command

Serving to http://127.0.0.1:7655/
[Ctrl-C to exit from terminal, or Ctrl-M i i to interrupt notebook kernel]


127.0.0.1 - - [12/Feb/2019 15:25:26] "GET / HTTP/1.1" 200 -



stopping Server...


In [37]:
(knockout_result.fluxes["G6PDH2r"], knockout_result.fluxes["PGI"])

(0.0, 10.0)

In [36]:
(
    pfba_result.fluxes["BIOMASS_Ec_iML1515_core_75p37M"],
    pfba_result.fluxes["G6PDH2r"],
    pfba_result.fluxes["PGI"],
)

(0.876997214426972, 2.3553662197924683, 7.644633780207531)

In [38]:
(lmoma_result.fluxes["G6PDH2r"], lmoma_result.fluxes["PGI"])

(0.0, 7.662083679805271)

## Dilution simulation

### Maximization of ATP maintenance

In [ ]:
with model:
    model.objective = "ATPM"
    # Simulate D 0.2 h-1
    model.reactions.BIOMASS_Ec_iML1515_core_75p37M.upper_bound = 0.21
    model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound = 0.19
    dilution_result = pfba(model)
    
    # Simulate Δzwf
    model.reactions.PGI.knock_out()
    dilution_knockout_result = pfba(model)
    dilution_lmoma_result = lmoma(model, reference=pfba_result.fluxes)

In [49]:
model.metabolites.glc__D_c.summary(solution=dilution_knockout_result)

PRODUCING REACTIONS -- D-Glucose (glc__D_c)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  ---------------------------------
100%      10  GLCt2pp   glc__D_p + h_p --> glc__D_c + h_c

CONSUMING REACTIONS -- D-Glucose (glc__D_c)
-------------------------------------------
%       FLUX  RXN ID    REACTION
----  ------  --------  ---------------------------------
100%      10  XYLI2     glc__D_c <=> fru_c


In [103]:
escher.Builder(
    map_name="e_coli_core.Core metabolism",
    model=model,
    reaction_data=dilution_result.fluxes.to_dict(),
).display_in_browser()


Note: You must interrupt the kernel to end this command

Serving to http://127.0.0.1:7655/
[Ctrl-C to exit from terminal, or Ctrl-M i i to interrupt notebook kernel]


127.0.0.1 - - [30/Jan/2019 14:33:14] "GET / HTTP/1.1" 200 -



stopping Server...


### Minimization of glucose consumption

In [4]:
# Minimize glucose consumption from medium - equiv to max exchange. 
model.objective = "EX_glc__D_e"
model.objective_direction = "max"
# make glucose uptake to be through Pts
model.reactions.GLCptspp.bounds = (0.0,1000)

# Simulate D 0.2 h-1
model.reactions.BIOMASS_Ec_iML1515_core_75p37M.bounds = (0.19,0.21)
# Glucose uptake rate should more or less match Ishii, 2007
# GUR for WT @ aerobic glucose = 2.87 mmol/h/gDW
model.reactions.GLCptspp.bounds = (2.86,2.88)

dilution_result = pfba(model)
all_data = pd.DataFrame()

In [5]:
# write WT information
df = pd.DataFrame(dilution_result.fluxes)
df.index = df.index.rename("ID")
df = df.reset_index()
df = df.assign(sample_id="WT")
        
# rename some columns to be more compatible with other simulations
df = df.assign(author = "COBRA")
df = df.assign(BiGG_ID = df.ID)
df = df.rename({"fluxes" : "flux"}, axis = 1)
        
# calculate normalized fluxes 
glucose_uptake = -1 * df[df['ID'] == 'EX_glc__D_e']['flux'].values[0]
df = df.assign(normalized_flux = lambda x: x.flux * 100 / glucose_uptake)

df.to_csv('../data/simulation_results/COBRA/WT_02.csv')

In [6]:
all_data = pd.concat([all_data, df])

In [7]:
for reaction in knockouts:
    with model:
        ko_gene = reaction["gene"]
        # Simulate knockout
        model.reactions.get_by_id(reaction["id"]).knock_out()
        lmoma_result = lmoma(model, reference=dilution_result.fluxes)

        df = pd.DataFrame(lmoma_result.fluxes)
        df.index = df.index.rename("ID")
        df = df.reset_index()
        df = df.assign(sample_id=reaction["gene"])
        
        # rename some columns to be more compatible with other simulations
        df = df.assign(author = "COBRA")
        df = df.assign(BiGG_ID = df.ID)
        df = df.rename({"fluxes" : "flux"}, axis = 1)
        
        # calculate normalized fluxes 
        glucose_uptake = -1 * df[df['ID'] == 'EX_glc__D_e']['flux'].values[0]
        df = df.assign(normalized_flux = lambda x: x.flux * 100 / glucose_uptake)
        
        df.to_csv(f"../data/simulation_results/COBRA/delta_{ko_gene}.csv")

        all_data = pd.concat([all_data, df], sort=False)

In [8]:
all_data.to_csv("../data/simulation_results/COBRA/knockouts_all.csv")

In [ ]:
escher.Builder(
    map_name="e_coli_core.Core metabolism",
    model=model,
    reaction_data=dilution_result.fluxes.to_dict(),
).display_in_browser()


Note: You must interrupt the kernel to end this command

Serving to http://127.0.0.1:7655/
[Ctrl-C to exit from terminal, or Ctrl-M i i to interrupt notebook kernel]


127.0.0.1 - - [12/Feb/2019 15:33:14] "GET / HTTP/1.1" 200 -
